In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import networkx as nx
from datetime import datetime

# T=pd.read_csv('rtv_50veh_first-run.csv')

def rtv_data_prep(trip_size,fleet_size,seed_yes,disk_engine):
    
    T = pd.read_sql_query('SELECT *' 'FROM data' , disk_engine)
    T = T.loc[T['size']<=trip_size]
#import requests and vehicle locations
    man_veh = pd.read_csv('Manhattan_dropoff_14_80000-80030.csv',index_col='index')
    seed = [99999,22842,42034,73027,38470,23843,59173,38533,10238,83799]
    if seed_yes>0:
        seed_value = seed[seed_yes-1]
        man_veh_50 = man_veh.sample(frac=fleet_size,random_state=seed_value)
        T = T.loc[T['veh_index'].isin(man_veh_50.index)]
    return(T)


In [20]:
def assign_data_prep(T):
    veh_rtv_filtered = T['veh_index'].unique().tolist()
    trip_rtv_filtered_str = T['request_index'].unique().tolist()
    trip_rtv_filtered = [int(x) for x in trip_rtv_filtered_str]
    unique_values_req =  pd.unique(T.loc[:,['r1','r2','r3','r4']].values.ravel()).tolist()
    unique_values_req = [int(x) for x in unique_values_req if (str(x) != 'nan' and x != None)]
    unique_values_req.sort()
    num_vehicles = len(veh_rtv_filtered) #vehicle
    return(veh_rtv_filtered,trip_rtv_filtered_str,trip_rtv_filtered,unique_values_req,num_vehicles)

In [4]:
def create_rtv_graph(unique_values_req,trip_rtv_filtered_str,T):
    RT_graph = nx.MultiGraph()
    for req in unique_values_req:
#     index_req = unique_values_req.index(req)
        RT_graph.add_node(req)
    for trip in trip_rtv_filtered_str:
#     index_trip = trip_rtv_filtered_str.index(trip)
        RT_graph.add_node(trip)
        trip_request_link = pd.unique(T.loc[T['request_index']==trip,['r1','r2','r3','r4']].values.ravel()).tolist()
        for trip_req in trip_request_link:
            if str(trip_req)!= 'nan':
#             index_req = unique_values_req.index(trip_req)
                RT_graph.add_edge(trip_req,trip)
    return(RT_graph)

In [5]:
def create_cost_matrix(num_vehicles,trip_rtv_filtered_str,T,penalty_constant,veh_rtv_filtered,unique_values_req):
    cost = np.empty((0,num_vehicles+1))
    for trip in trip_rtv_filtered_str:
        cost_sub = []
        for vehicle in veh_rtv_filtered:
            vehicle_trips = T.loc[T['veh_index']==vehicle]
            if trip in vehicle_trips['request_index'].values:
                delay = vehicle_trips.loc[vehicle_trips['request_index']==trip].delay.values[0]
            else:
                delay = 0
            cost_sub.append(delay)
        #add trip cost of the dummy vehicle in the last column
        cost_sub.append(0)

    #     cost.append(cost_sub)
        cost = np.append(cost,[cost_sub],axis=0)

    #Add cost of rejecting request
    cost_reject = []
    for veh in veh_rtv_filtered:
        cost_reject.append(0)
    cost_reject.append(penalty_constant)
    for request in unique_values_req:
        cost = np.append(cost,[cost_reject],axis=0)
    return(cost)

In [6]:
from ortools.linear_solver import pywraplp
# Create the mip solver with gurobi
def create_solver():
    solver = pywraplp.Solver('SolveIntegerProblem',
                               pywraplp.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING)
    return(solver)

count_obj = 0
num_trips = len(trip_rtv_filtered) #trip

# x[i, j] is an array of 0-1 variables, which will be 1
# if trip i is assigned to vehicle j.
x = np.empty((num_trips+len(unique_values_req),num_vehicles+1),dtype=object)
for i in range(num_trips):
    for j in range(num_vehicles):
        if T.loc[T['request_index']==trip_rtv_filtered_str[i]].veh_index.values[0] == veh_rtv_filtered[j]:
            x[i, j] = solver.IntVar(0, 1, '')
            count_obj+=1
        else:
            x[i, j] = 0
    #add dummy vehicle at the last column
    x[i,num_vehicles] = 0
# add rejected trip for each request at the last rows
for i in range(len(unique_values_req)):
    for j in range(num_vehicles):
        x[num_trips+i,j] = 0
    # add binary variable for the rejected trip, which allow it to be paired only with the dummy vehicle 
    x[num_trips+i,num_vehicles] = solver.IntVar(0, 1, '')
    count_obj+=1

In [7]:
def create_x_matrix(trip_rtv_filtered,cost,solver):    
    num_trips = len(trip_rtv_filtered) #trip
    x = np.empty((cost.shape[0],cost.shape[1]),dtype=object)
    for i in range(cost.shape[0]):
        for j in range(cost.shape[1]):
            if cost[i,j]>0:
                x[i,j] = solver.IntVar(0, 1, '')
            else:
                x[i,j] = 0
    return(x,num_trips)

In [8]:
def create_req_matrix(unique_values_req,num_trips,RT_graph,trip_rtv_filtered_str,num_vehicles,cost,x):    
    num_req = len(unique_values_req)
    r = np.empty((num_req,num_trips+num_req),dtype=object)
    for i in range(num_req):
        trip_index_all = []
        #get all edges connecting to a request and append to a list
        for edge in RT_graph.edges(unique_values_req[i]):
            trip_index = trip_rtv_filtered_str.index(edge[1])
            trip_index_all.append(trip_index)
        for j in range(num_trips):
            if j in trip_index_all:
                trip_req = []
                for n in range(num_vehicles+1):
                    if cost[j,n]>0:
                        trip_req.append(n)
                r[i, j] = sum([x[j, k] for k in trip_req])
            else:
                r[i, j] = 0
    for i in range(num_req):
        for j in range(num_req):
            if i==j:
                r[i,num_trips+j] = x[num_trips+i, num_vehicles] 
            else:
                r[i,num_trips+j] = 0
    return(r,num_req)

num_req = len(unique_values_req)
r = np.empty((num_req,num_trips+num_req),dtype=object)

for i in range(num_req):
    req_index = unique_values_req[i]
    for j in range(num_trips):
        trip_index = trip_rtv_filtered_str[j]
        request_list = pd.unique(T.loc[T['request_index']==trip_index,['r1','r2','r3','r4']].values.ravel())
        if req_index in request_list:
            trip_req = []
            for n in range(num_vehicles+1):
                if cost[j,n]>0:
                    trip_req.append(n)
            r[i, j] = sum([x[j, k] for k in trip_req])
            count_obj+=1
        else:
            r[i, j] = 0
    #add rejected trip
#     r[i,num_trips] = sum([x[num_trips+i, k] for k in range(num_vehicles+1)])
    
for i in range(num_req):
    for j in range(num_req):
        if i==j:
            r[i,num_trips+j] = x[num_trips+i, num_vehicles] 
        else:
            r[i,num_trips+j] = 0

In [9]:
def define_constraints(x,num_req,num_trips,num_vehicles,penalty_constant,cost,unique_values_req,solver,r):
    # Each trip is assigned to at most 1 vehicle. 
    for i in range(x.shape[0]):
        solver.Add(solver.Sum([x[i, j] for j in range(x.shape[1])]) <= 1)

    # Each vehicle is assigned to at most one trip. 
    for j in range(x.shape[1]-1):
        solver.Add(solver.Sum([x[i, j] for i in range(x.shape[0])]) <= 1)

    # Each request is assigned to exactly 1 trip (including rejected trip).
    for i in range(num_req):
        solver.Add(solver.Sum([r[i, j] for j in range(r.shape[1])]) == 1)

    objective_terms = []
    #add cost of each trip 
    for i in range(num_trips):
        for j in range(num_vehicles):
            if cost[i,j]>0 and cost[i,j]<penalty_constant:
                objective_terms.append(cost[i,j] * x[i, j])
    #add cost of rejection 
    for i in range(len(unique_values_req)):
        objective_terms.append(cost[num_trips+i][num_vehicles] * x[num_trips+i,num_vehicles])
    return(objective_terms)

In [10]:
def solve_mip(objective_terms,solver):
    solver.Minimize(solver.Sum(objective_terms))
    status = solver.Solve()
    return(status)

In [11]:
def print_solution(seed_no,solver,Assignment_ILP,status,num_trips,num_req,num_vehicles,x,trip_rtv_filtered_str,veh_rtv_filtered,cost,unique_values_req):
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total Delay = ', solver.Objective().Value(),' Seed no.= ',seed_no)
        for i in range(num_trips + num_req):
            for j in range(num_vehicles):
                if not isinstance(x[i, j],int):
                    if x[i,j].solution_value() > 0.2:
#                         print('Trip %d assigned to vehicle %d.  Delay = %d' %
#                               (i, j, cost[i][j]))
                        Assignment_ILP = Assignment_ILP.append({'trip':trip_rtv_filtered_str[i],'vehicle':veh_rtv_filtered[j],'delay':cost[i][j],'solution_val':x[i, j].solution_value(),'seed_no':seed_no},ignore_index=True)
            if not isinstance(x[i, j+1],int) and x[i,j+1].solution_value() > 0.2:
#                 print('Trip %d assigned to vehicle %d.  Delay = %d' %
#                               (i, j+1, cost[i][j+1]))
                Assignment_ILP = Assignment_ILP.append({'trip':unique_values_req[i-num_trips],'vehicle':'rejected','delay':cost[i][j+1],'solution_val':x[i, j+1].solution_value(),'seed_no':seed_no},ignore_index=True)
    return(Assignment_ILP)

In [12]:
def execute_all(trip_size,fleet_size,disk_engine,penalty_constant):
    print('Trip size = ',trip_size,' Fleet size = ', fleet_size*100,'%')
    Assignment_ILP = pd.DataFrame()
    if fleet_size < 1:
        for i in range(10):
            T = rtv_data_prep(trip_size,fleet_size,i+1,disk_engine)
            veh_rtv_filtered,trip_rtv_filtered_str,trip_rtv_filtered,unique_values_req,num_vehicles = assign_data_prep(T)
            RT_graph = create_rtv_graph(unique_values_req,trip_rtv_filtered_str,T)
            cost = create_cost_matrix(num_vehicles,trip_rtv_filtered_str,T,penalty_constant,veh_rtv_filtered,unique_values_req)
            solver = create_solver()
            x,num_trips = create_x_matrix(trip_rtv_filtered,cost,solver)
            r,num_req = create_req_matrix(unique_values_req,num_trips,RT_graph,trip_rtv_filtered_str,num_vehicles,cost,x)
            objective_terms = define_constraints(x,num_req,num_trips,num_vehicles,penalty_constant,cost,unique_values_req,solver,r)
            status = solve_mip(objective_terms,solver)
            Assignment_ILP = print_solution(i+1,solver,Assignment_ILP,status,num_trips,num_req,num_vehicles,x,trip_rtv_filtered_str,veh_rtv_filtered,cost,unique_values_req)
    else:
        T = rtv_data_prep(trip_size,fleet_size,0,disk_engine)
        veh_rtv_filtered,trip_rtv_filtered_str,trip_rtv_filtered,unique_values_req,num_vehicles = assign_data_prep(T)
        RT_graph = create_rtv_graph(unique_values_req,trip_rtv_filtered_str,T)
        cost = create_cost_matrix(num_vehicles,trip_rtv_filtered_str,T,penalty_constant,veh_rtv_filtered,unique_values_req)
        solver = create_solver()
        x,num_trips = create_x_matrix(trip_rtv_filtered,cost,solver)
        r,num_req = create_req_matrix(unique_values_req,num_trips,RT_graph,trip_rtv_filtered_str,num_vehicles,cost,x)
        objective_terms = define_constraints(x,num_req,num_trips,num_vehicles,penalty_constant,cost,unique_values_req,solver,r)
        status = solve_mip(objective_terms,solver)
        Assignment_ILP = print_solution(0,solver,Assignment_ILP,status,num_trips,num_req,num_vehicles,x,trip_rtv_filtered_str,veh_rtv_filtered,cost,unique_values_req)

    return(Assignment_ILP)

In [21]:
import os
penalty_constant = 10000000
path = "C:\\Users\\Chonlavit\\OneDrive\\2019-Leeds Doc\\TRAN5911_disser\\Gurobi\\Shared-mobility-Route-optimization\\ILP_results_v2\\"
disk_engine = create_engine('sqlite:///T_wait-3_delay-5_v2.db')
for trip_size in [1,2,4]:
    for fleet_size in [0.25,0.5,0.75,1]:
        waiting_t = 3
        dropoff_t = 5
        now1 = datetime.now()
        print(waiting_t," ",dropoff_t," ",fleet_size," ",trip_size)
        result_ilp_df = execute_all(trip_size,fleet_size,disk_engine,penalty_constant)
        result_ilp_df['trip_size'] = trip_size
        result_ilp_df['fleet_size'] = fleet_size*100
        result_ilp_df['waiting_t_constr'] = waiting_t
        result_ilp_df['dropoff_t_constr'] = dropoff_t
        name_f = 'ass_veh-'+str(fleet_size*100)+ '_wait-' + str(waiting_t) + '_delay-'+ str(dropoff_t) + '_capa-'+ str(trip_size) +'.csv' 
        result_ilp_df.to_csv(os.path.join(path,name_f))
        now2 = datetime.now()
        if fleet_size<1:
            print((now2-now1)/10)
        else:
            print(now2-now1)

3   5   0.25   1
Trip size =  1  Fleet size =  25.0 %
Total Delay =  1210002043.0  Seed no.=  1
Total Delay =  1170002201.0  Seed no.=  2
Total Delay =  1260002261.0  Seed no.=  3
Total Delay =  1260001976.0  Seed no.=  4
Total Delay =  1350002038.0  Seed no.=  5
Total Delay =  1220002206.0  Seed no.=  6
Total Delay =  1090002778.0  Seed no.=  7
Total Delay =  1200002537.0  Seed no.=  8
Total Delay =  1210002038.0  Seed no.=  9
Total Delay =  1220001806.0  Seed no.=  10
0:00:03.344126
3   5   0.5   1
Trip size =  1  Fleet size =  50.0 %
Total Delay =  870004651.0  Seed no.=  1
Total Delay =  860005328.0  Seed no.=  2
Total Delay =  920005529.0  Seed no.=  3
Total Delay =  890004606.0  Seed no.=  4
Total Delay =  960005215.0  Seed no.=  5
Total Delay =  940005289.0  Seed no.=  6
Total Delay =  870005185.0  Seed no.=  7
Total Delay =  870005164.0  Seed no.=  8
Total Delay =  970004718.0  Seed no.=  9
Total Delay =  950005590.0  Seed no.=  10
0:00:06.220549
3   5   0.75   1
Trip size =  1

In [22]:
now1 = datetime.now()
trip_size = 4 #1-4
fleet_size = 0.50 #0-1
disk_engine = create_engine('sqlite:///T_wait-3_delay-5.db')
penalty_constant = 1000000
result_ilp_df = execute_all(trip_size,fleet_size,disk_engine,penalty_constant)
now2 = datetime.now()
print(now2-now1)

Trip size =  4  Fleet size =  50.0 %
Total Delay =  3034001.0  Seed no.=  1
Total Delay =  5029891.0  Seed no.=  2
Total Delay =  6030773.0  Seed no.=  3
Total Delay =  1028627.0  Seed no.=  4
Total Delay =  6029238.0  Seed no.=  5
Total Delay =  5031322.0  Seed no.=  6
Total Delay =  28498.0  Seed no.=  7
Total Delay =  2028318.0  Seed no.=  8
Total Delay =  6034058.0  Seed no.=  9
Total Delay =  12029724.0  Seed no.=  10
0:51:11.094048


In [13]:
trip_size = 2 #1-4
fleet_size = 0.25 #0-1
disk_engine = create_engine('sqlite:///T_wait-3_delay-5.db')
penalty_constant = 10000000
result_ilp_df = execute_all(trip_size,fleet_size,disk_engine,penalty_constant)
import os
path = "C:\\Users\\Chonlavit\\OneDrive\\2019-Leeds Doc\\TRAN5911_disser\\Gurobi\\Shared-mobility-Route-optimization\\ILP_results\\"

result_ilp_df.to_csv(os.path.join(path,r'ass_veh-25_wait-3_delay-5_capa-2.csv'))

Trip size =  2  Fleet size =  25.0 %
Total Delay =  770009916.0  Seed no.=  1
Total Delay =  730010905.0  Seed no.=  2
Total Delay =  750010038.0  Seed no.=  3
Total Delay =  800009577.0  Seed no.=  4
Total Delay =  930009329.0  Seed no.=  5
Total Delay =  790010418.0  Seed no.=  6
Total Delay =  690011666.0  Seed no.=  7
Total Delay =  760011525.0  Seed no.=  8
Total Delay =  780008883.0  Seed no.=  9
Total Delay =  800009716.0  Seed no.=  10


In [ ]:
import multiprocessing

In [ ]:
T.rename(columns={'size':'trip_size'}, inplace=True)
Assignment_ILP_norej =  Assignment_ILP.loc[Assignment_ILP['vehicle']!='rejected']
for index,row in Assignment_ILP_norej.iterrows():
    Assignment_ILP_norej.loc[index,'r1'] = T.loc[T['request_index']==row['trip']].r1.values[0]
    Assignment_ILP_norej.loc[index,'r2'] = T.loc[T['request_index']==row['trip']].r2.values[0]
    Assignment_ILP_norej.loc[index,'r3'] = T.loc[T['request_index']==row['trip']].r3.values[0]
    Assignment_ILP_norej.loc[index,'r4'] = T.loc[T['request_index']==row['trip']].r4.values[0]
    Assignment_ILP_norej.loc[index,'size'] = T.loc[T['request_index']==row['trip']].trip_size.values[0]

In [ ]:
Assignment_ILP_norej =  Assignment_ILP.loc[Assignment_ILP['vehicle']!='rejected']
T_assigned = T.loc[T['request_index'].isin(Assignment_ILP_norej['trip'].values.tolist())]

In [ ]:
T_assigned['size'].sum()

In [53]:
result_ilp_df

,delay,seed_no,solution_val,trip,vehicle
0,86.0,1.0,1.0,22337,290948
1,60.0,1.0,1.0,350249,1410
2,84.0,1.0,1.0,222333,145801
3,63.0,1.0,1.0,6087,175845
4,142.0,1.0,1.0,46882,70588
...,...,...,...,...,...
1767,10000000.0,10.0,1.0,357506,rejected
1768,10000000.0,10.0,1.0,360488,rejected
1769,10000000.0,10.0,1.0,364390,rejected
1770,10000000.0,10.0,1.0,375402,rejected


In [20]:
%timeit -n1 -r1 print('1') 

1
339 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [33]:
now2 = datetime.now()

In [34]:
print(now2-now1)

0:00:11.745489


In [29]:
now2-now

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [3]:
disk_engine = create_engine('sqlite:///T_wait-3_delay-5.db')
TT = pd.read_sql_query('SELECT *' 'FROM data' , disk_engine)

In [4]:
TT.shape

(39983, 11)

In [14]:
disk_engine = create_engine('sqlite:///T_wait-3_delay-5_v2.db')
T = pd.read_sql_query('SELECT *' 'FROM data' , disk_engine)

In [17]:
T['r4'].value_counts()

348888    101
330826     35
279563     24
298779     13
148795     11
278545      7
253002      7
281191      7
379463      5
148200      4
309957      3
337896      2
344719      2
197460      2
350249      2
321321      2
374853      2
298527      1
249519      1
353331      1
Name: r4, dtype: int64